In [ ]:
!unzip asr.zip

Archive:  asr.zip
   creating: asr/
  inflating: asr/.DS_Store           
  inflating: __MACOSX/asr/._.DS_Store  
  inflating: asr/requirements.txt    
  inflating: asr/train_model.py      
  inflating: asr/download_data.py    
   creating: asr/conf/
   creating: asr/src/
  inflating: asr/conf/conformer_ctc.yaml  
  inflating: asr/conf/conformer_ctc_wide.yaml  
  inflating: asr/conf/ebranchformer_ctc.yaml  
  inflating: asr/conf/conformer_las.yaml  
  inflating: asr/src/metrics.py      
  inflating: asr/src/transforms.py   
  inflating: asr/src/.DS_Store       
  inflating: __MACOSX/asr/src/._.DS_Store  
  inflating: asr/src/__init__.py     
   creating: asr/src/models/
   creating: asr/src/encoders/
   creating: asr/src/submodules/
   creating: asr/src/decoders/
  inflating: asr/src/optim.py        
  inflating: asr/src/data.py         
  inflating: asr/src/models/ctc_model.py  
  inflating: asr/src/models/__init__.py  
  inflating: asr/src/models/las_model.py  
  inflating: asr/src/e

In [5]:
!pip install pytorch_lightning hydra-core

In [6]:
cd asr

/content/asr


In [7]:
from tqdm.notebook import tqdm
import shutil
import asr.download_data
import importlib
importlib.reload(asr.download_data)
from asr.download_data import download_from_gdrive
from asr.src.metrics import WER
from asr.src.models.ctc_model import CTCModel
from omegaconf import OmegaConf
import pytorch_lightning as pl
import pandas as pd
import torch

def calculate_greedy_wer(preds_list, encoded_len_list, targets_list):
    metric = WER()
    refs = []
    hyps = []
    for pred_batch, encoded_len_batch, targets_batch in tqdm(zip(preds_list, encoded_len_list, targets_list), total=len(preds_list)):
        for pred, pred_len, target in zip(pred_batch, encoded_len_batch, targets_batch):
            hyps.append(model.decoder.decode_hypothesis(pred[:pred_len], unique_consecutive=True))
            refs.append(model.decoder.decode_hypothesis(target, unique_consecutive=False))

    metric.update(refs, hyps)
    wer = metric.compute()
    return wer[0]

In [ ]:
download_from_gdrive()

Downloading...
From (original): https://drive.google.com/uc?id=1iQd89RCOuGryVoDzyWCnSulaj4kcm-I8
From (redirected): https://drive.google.com/uc?id=1iQd89RCOuGryVoDzyWCnSulaj4kcm-I8&confirm=t&uuid=6041d1e0-9c43-4ff0-882b-5a68f00c6b97
To: /content/asr/asr_data.tar
100%|██████████| 2.78G/2.78G [00:39<00:00, 69.8MB/s]


In [ ]:
shutil.unpack_archive("asr_data.tar", "./data")

In [ ]:
!python asr/train_model.py --config-name ebranchformer_ctc

/content/asr/train_model.py:24: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="conf", config_name="conformer_ctc")
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
model.parameters: 11266850
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2024-10-20 18:12:38.510838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 18:12:38.534628: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN 

In [ ]:
!python asr/train_model.py --config-name ebranchformer_ctc init_weights=/content/asr/data/checkpoints_and_tokenizer/ebranchformer_ckpt.ckpt

/content/asr/train_model.py:24: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="conf", config_name="conformer_ctc")
/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
model.parameters: 11266850
/content/asr/train_model.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` wi

In [ ]:
cfg = OmegaConf.load("conf/ebranchformer_ctc.yaml")
model = CTCModel(cfg)
model.eval()
model.freeze()

ckpt = torch.load('data/checkpoints_and_tokenizer/ebranchformer_ckpt.ckpt', map_location="cpu")
model.load_state_dict(ckpt)
model.to('cpu')

<ipython-input-79-1c1ddbaa0213>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('data/checkpoints_and_tokenizer/ebranchformer_ckpt.ckpt', map_location="cp

CTCModel(
  (encoder): EBranchformerEncoder(
    (subsampling): StackingSubsampling(
      (out): Linear(in_features=256, out_features=256, bias=True)
    )
    (pos_embedding): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0-7): 8 x EBranchformerEncoderLayer(
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (cgmlp): ConvolutionalGatingMLP(
          (channel_proj1): Sequential(
            (0): Linear(in_features=256, out_features=768, bias=True)
            (1): GELU(approximate='none')
          )
          (csgu): ConvolutionalSpatialGatingUnit(
            (norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
            (conv): Conv1d(384, 384, kernel_size=(7,), stride=(1,), padding=(3,), groups=384)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (channel_proj2): Sequential(
       

In [ ]:
results = {
    "features": [],
    "features_len": [],
    "targets": [],
    "target_len": [],
    "logprobs": [],
    "encoded_len": [],
    "preds": []
}

for i, batch in tqdm(enumerate(model.val_dataloader()), desc="Processing Batches"):
    features, features_len, targets, target_len = batch

    with torch.inference_mode():
        logprobs, encoded_len, preds = model(features.to('cpu'), features_len.to('cpu'))

    results["features"].append(features)
    results["features_len"].append(features_len)
    results["targets"].append(targets)
    results["target_len"].append(target_len)
    results["logprobs"].append(logprobs)
    results["encoded_len"].append(encoded_len)
    results["preds"].append(preds)

    if i == 4:
        break

Processing Batches: 0it [00:00, ?it/s]

In [ ]:
wer = calculate_greedy_wer(results["preds"],
                           results["encoded_len"],
                           results["targets"])
print(f"WER on {cfg['val_dataloader']['dataset']['manifest_name']}: {wer}")

  0%|          | 0/5 [00:00<?, ?it/s]

WER on test_opus/crowd/manifest.jsonl: 0.9695273637771606


In [ ]:
cfg['val_dataloader']['dataset']['manifest_name'] = 'test_opus/farfield/manifest.jsonl'

cfg = OmegaConf.load("conf/ebranchformer_ctc.yaml")
model = CTCModel(cfg)
model.eval()
model.freeze()

ckpt = torch.load('data/checkpoints_and_tokenizer/ebranchformer_ckpt.ckpt', map_location="cpu")
model.load_state_dict(ckpt)
model.to('cpu')

<ipython-input-91-d2d1b83f01da>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('data/checkpoints_and_tokenizer/ebranchformer_ckpt.ckpt', map_location="cp

CTCModel(
  (encoder): EBranchformerEncoder(
    (subsampling): StackingSubsampling(
      (out): Linear(in_features=256, out_features=256, bias=True)
    )
    (pos_embedding): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0-7): 8 x EBranchformerEncoderLayer(
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (cgmlp): ConvolutionalGatingMLP(
          (channel_proj1): Sequential(
            (0): Linear(in_features=256, out_features=768, bias=True)
            (1): GELU(approximate='none')
          )
          (csgu): ConvolutionalSpatialGatingUnit(
            (norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
            (conv): Conv1d(384, 384, kernel_size=(7,), stride=(1,), padding=(3,), groups=384)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (channel_proj2): Sequential(
       

In [ ]:
results = {
    "features": [],
    "features_len": [],
    "targets": [],
    "target_len": [],
    "logprobs": [],
    "encoded_len": [],
    "preds": []
}

for i, batch in tqdm(enumerate(model.val_dataloader()), desc="Processing Batches"):
    features, features_len, targets, target_len = batch

    with torch.inference_mode():
        logprobs, encoded_len, preds = model(features.to('cpu'), features_len.to('cpu'))

    results["features"].append(features)
    results["features_len"].append(features_len)
    results["targets"].append(targets)
    results["target_len"].append(target_len)
    results["logprobs"].append(logprobs)
    results["encoded_len"].append(encoded_len)
    results["preds"].append(preds)

    if i == 4:
        break

wer = calculate_greedy_wer(results["preds"],
                           results["encoded_len"],
                           results["targets"])
print(f"WER on {cfg['val_dataloader']['dataset']['manifest_name']}: {wer}")

Processing Batches: 0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

WER on test_opus/farfield/manifest.jsonl: 0.8912839889526367


In [ ]:
manifest_path = 'data/train_opus/manifest.jsonl'
manifest_df = pd.read_json(manifest_path, lines=True)

sample_size = 100
sample_df = manifest_df.sample(n=sample_size, random_state=42)

cpu_manifest_path = 'data/train_opus/cpu_manifest.jsonl'
sample_df.to_json(cpu_manifest_path, orient='records', lines=True)

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 9.6 GB  |     Proc size: 3.0 GB
GPU RAM Free: 59MB | Used: 15043MB | Util  98% | Total     15360MB


In [ ]:
cfg = OmegaConf.load("conf/ebranchformer_ctc.yaml")
cfg['train_dataloader']['dataset']['manifest_name'] = 'train_opus/cpu_manifest.jsonl'
cfg['val_dataloader']['dataset']['manifest_name'] = 'train_opus/cpu_manifest.jsonl'

cfg['train_dataloader']['batch_size'] = 8
cfg['val_dataloader']['batch_size'] = 8
cfg.trainer['accumulate_grad_batches'] = 4

model = CTCModel(cfg)
trainer = pl.Trainer(
    logger=pl.loggers.TensorBoardLogger(save_dir="logs"),
    **cfg.trainer
)

trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type                 | Params | Mode 
----------------------------------------------------------
0 | encoder  | EBranchformerEncoder | 11.3 M | train
1 | decoder  | ConvDecoder          | 8.7 K  | train
2 | wer      | WER                  | 0      | train
3 | ctc_loss | CTCLoss              | 0      | train
----------------------------------------------------------
10.0 M    Trainable params
1.3 M     Non-trainable params
11.3 M    Total params
45.067    Total estimated model params size (MB)
260       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=1000). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
cfg = OmegaConf.load("conf/ebranchformer_ctc.yaml")
cfg['train_dataloader']['dataset']['manifest_name'] = 'train_opus/cpu_manifest.jsonl'
cfg['val_dataloader']['dataset']['manifest_name'] = 'train_opus/cpu_manifest.jsonl'

ckpt = torch.load('logs/lightning_logs/version_1/checkpoints/epoch=1-step=8.ckpt',
                  map_location="cpu")
model = CTCModel(cfg)
model.load_state_dict(ckpt['state_dict'])
model.to('cpu')

model = CTCModel(cfg)
trainer = pl.Trainer(
    logger=pl.loggers.TensorBoardLogger(save_dir="logs"),
    **cfg.trainer
)

trainer.fit(model)

<ipython-input-16-7f4d7a3b657d>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('logs/lightning_logs/version_1/checkpoints/epoch=1-step=8.ckpt',
INFO:pyto

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=1000). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

In [8]:
cfg = OmegaConf.load("conf/ebranchformer_ctc.yaml")
cfg['val_dataloader']['dataset']['manifest_name'] = 'train_opus/cpu_manifest.jsonl'
model = CTCModel(cfg)
model.eval()
model.freeze()

results = {
    "features": [],
    "features_len": [],
    "targets": [],
    "target_len": [],
    "logprobs": [],
    "encoded_len": [],
    "preds": []
}

for i, batch in tqdm(enumerate(model.val_dataloader()), desc="Processing Batches"):
    features, features_len, targets, target_len = batch

    with torch.inference_mode():
        logprobs, encoded_len, preds = model(features.to('cpu'), features_len.to('cpu'))

    results["features"].append(features)
    results["features_len"].append(features_len)
    results["targets"].append(targets)
    results["target_len"].append(target_len)
    results["logprobs"].append(logprobs)
    results["encoded_len"].append(encoded_len)
    results["preds"].append(preds)

    if i == 4:
        break

wer = calculate_greedy_wer(results["preds"],
                           results["encoded_len"],
                           results["targets"])
print(f"WER on random weights: {wer}")

Processing Batches: 0it [00:00, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

WER on train_opus/cpu_manifest.jsonl: 1.0105078220367432


In [9]:
ckpt = torch.load('logs/lightning_logs/version_1/checkpoints/epoch=1-step=8.ckpt', map_location="cpu")
model.load_state_dict(ckpt['state_dict'])
model.to('cpu')

results = {
    "features": [],
    "features_len": [],
    "targets": [],
    "target_len": [],
    "logprobs": [],
    "encoded_len": [],
    "preds": []
}

for i, batch in tqdm(enumerate(model.val_dataloader()), desc="Processing Batches"):
    features, features_len, targets, target_len = batch

    with torch.inference_mode():
        logprobs, encoded_len, preds = model(features.to('cpu'), features_len.to('cpu'))

    results["features"].append(features)
    results["features_len"].append(features_len)
    results["targets"].append(targets)
    results["target_len"].append(target_len)
    results["logprobs"].append(logprobs)
    results["encoded_len"].append(encoded_len)
    results["preds"].append(preds)

    if i == 4:
        break

wer = calculate_greedy_wer(results["preds"],
                           results["encoded_len"],
                           results["targets"])
print(f"WER on 3 epochs: {wer}")

<ipython-input-9-6f07d8ab8bc2>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('logs/lightning_logs/version_1/checkpoints/epoch=1-step=8.ckpt', map_locati

Processing Batches: 0it [00:00, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

WER on 3 epochs: 1.007005214691162
